In [1]:
import glob
import os
import sys

sys.path.append("./src")
try:
    sys.path.append(
        glob.glob(
            "%s/PythonAPI/carla/dist/carla-*%d.%d-%s.egg"
            % (
                os.environ.get("CARLA_ROOT", "."),
                sys.version_info.major,
                sys.version_info.minor,
                "win-amd64" if os.name == "nt" else "linux-x86_64",
            )
        )[0]
    )
except IndexError:
    pass

In [2]:
import carla
from replay_parser import Replay

### Initialize client and specify the replay file

In [3]:
# Carla server will handle the decoding of the log file (to string)
# If the replay is recorded with different version of Carla, errors may occur

client = carla.Client("localhost", 2000)
client.set_timeout(5.0)

# Note: Must provide absolute path to the log file
filepath = os.path.abspath("./examples/data/Town01_record.dat")

### Parsing the replay file

In [4]:
# By default, the replay will not parse all information immediately.
# we will only parse what you need, when you call it.
# In this script we will parse all information immediately.

replay = Replay(client, filepath, lazy_init=False)

#### Episode information

In [5]:
# From the replay file we can get the map name, duration of the record, 
print(replay.map_name, replay.duration)

Town01 11.6


#### Actors

In [6]:
# We provide access to all actors that are "spawned" in the world
# Classify them into different types
num_vehicles = len(replay.vehicles)
num_walkers = len(replay.walkers)
num_sensors = len(replay.sensors)
num_obstacles = len(replay.obstacles)

# Sensor is not considered as an "actor" since it cannot perform "action"
print(len(replay.actors) == num_vehicles + num_walkers + num_obstacles)

True


In [7]:
# There is a inner function called `_parse_actors` which is used to parse the actor information
# If you need some other type of actors (for example TrafficLight), you can do:

lights = replay._parse_actors("traffic.traffic_light")
print(len(lights))

36


In [8]:
# Each actor is a dataclass holding its "id", "blueprint", "spawn_point" and "attributes"
# If the actor have some sensors attached, it will also have a "sensors" attribute

print(replay.vehicles[0])

Actor(id=198, blueprint='vehicle.tesla.model3', spawn_point=Location(x=-1.94737, y=24.961199999999998, z=0.00142345), attributes={'generation': '1', 'number_of_wheels': '4', 'sticky_control': 'true', 'object_type': '', 'color': '17,37,103', 'role_name': 'hero'}, sensors=[Actor(id=199, blueprint='sensor.other.collision', spawn_point=Location(x=-1.94737, y=24.961199999999998, z=0.00142345), attributes={'role_name': 'front'}, sensors=[]), Actor(id=200, blueprint='sensor.other.lane_invasion', spawn_point=Location(x=-1.94737, y=24.961199999999998, z=0.00142345), attributes={'role_name': 'front'}, sensors=[])])


In [9]:
# You can retrieve the actor by its id or rolename

replay.get_actor_by_id(198) == replay.get_actor_by_rolename("hero")

True

In [10]:
# Support filter actors by their blueprint (wildcard is supported)

replay.filter_actor_by_blueprint("vehicle.*")

[Actor(id=198, blueprint='vehicle.tesla.model3', spawn_point=Location(x=-1.94737, y=24.961199999999998, z=0.00142345), attributes={'generation': '1', 'number_of_wheels': '4', 'sticky_control': 'true', 'object_type': '', 'color': '17,37,103', 'role_name': 'hero'}, sensors=[Actor(id=199, blueprint='sensor.other.collision', spawn_point=Location(x=-1.94737, y=24.961199999999998, z=0.00142345), attributes={'role_name': 'front'}, sensors=[]), Actor(id=200, blueprint='sensor.other.lane_invasion', spawn_point=Location(x=-1.94737, y=24.961199999999998, z=0.00142345), attributes={'role_name': 'front'}, sensors=[])]),
 Actor(id=201, blueprint='vehicle.lincoln.mkz_2020', spawn_point=Location(x=-1.9299199999999999, y=79.2626, z=-0.00502571), attributes={'generation': '2', 'number_of_wheels': '4', 'sticky_control': 'true', 'driver_id': '0', 'object_type': '', 'color': '0,0,0', 'role_name': 'car1'}, sensors=[Actor(id=202, blueprint='sensor.other.collision', spawn_point=Location(x=-1.9299199999999999,

#### Frames

In [11]:
# The replay file is recorded frame by frame (each `world.tick()`)

print(len(replay.frame_info))

233


In [12]:
# Carla will only record minimal information that can allow it to play back
# Actor is moving around by teleporting, all physical information is not recorded (like velocity, acceleration, etc.)
# But we calculated these information for you

first_frame = replay.get_frame(1)   # == replay.frame_info[0]
print("frame_id:", first_frame.frame_id)
print("frame_time:", first_frame.frame_time)
print("transform:", first_frame.transform)
print("velocity:", first_frame.velocity)
print("acceleration:", first_frame.acceleration)
print("action:", first_frame.action)

frame_id: 1
frame_time: 0.0
transform: {198: Transform(location=Location(x=-1.94737, y=24.961199999999998, z=0.00137814), rotation=Rotation(pitch=0.0, yaw=0.0, roll=90.0221)), 201: Transform(location=Location(x=-1.9299199999999999, y=79.2626, z=-0.00502571), rotation=Rotation(pitch=0.0, yaw=0.0, roll=90.0221)), 204: Transform(location=Location(x=2.1247, y=232.456, z=-0.005214079999999999), rotation=Rotation(pitch=0.0, yaw=0.0, roll=-90.0299)), 207: Transform(location=Location(x=2.10463, y=212.27, z=-0.00527058), rotation=Rotation(pitch=-0.000671387, yaw=0.000662528, roll=-90.0299)), 210: Transform(location=Location(x=-2.0758, y=65.278, z=-0.00531578), rotation=Rotation(pitch=-0.000152588, yaw=0.000198075, roll=90.022))}
velocity: {198: Vector3D(x=0, y=0, z=0), 201: Vector3D(x=0, y=0, z=0), 204: Vector3D(x=0, y=0, z=0), 207: Vector3D(x=0, y=0, z=0), 210: Vector3D(x=0, y=0, z=0)}
acceleration: {198: Vector3D(x=0, y=0, z=0), 201: Vector3D(x=0, y=0, z=0), 204: Vector3D(x=0, y=0, z=0), 207:

In [13]:
# You can also get an actor-wise view instead
# pass `use_carla_api=True` to get Carla instances

actor_info = first_frame.get_info_by_id(201, use_carla_api=False)
print("Actor id:", actor_info.id)
print("Actor transform:", actor_info.transform)
print("Actor velocity:", actor_info.velocity)
print("Actor acceleration:", actor_info.acceleration)
print("Actor action:", actor_info.action)

Actor id: 201
Actor transform: Transform(location=Location(x=-1.9299199999999999, y=79.2626, z=-0.00502571), rotation=Rotation(pitch=0.0, yaw=0.0, roll=90.0221))
Actor velocity: Vector3D(x=0, y=0, z=0)
Actor acceleration: Vector3D(x=0, y=0, z=0)
Actor action: Action(steering=0.0, throttle=0.0, brake=0.0, handbrake=False, gear=0, speed=None, type='vehicle')


#### Collision info

In [ ]:
# There is very little information about the collision info in the replay file
# We can only know at which frame the collision happens, and the id of the actors
# Only actor that attached with a collision sensor will record the collision information

print(*replay.get_collision_frames()[:5], sep="\n")

#### Trajectory

In [ ]:
# We provide a function to return the trajectory of an actor
# The trajectory is a list of `Transform`

traj = replay.get_actor_trajectory(201)
print(*traj[:5], sep="\n")

# Note: 
# 1. The first frame's location may not be the same as the spawn point
# 2. We removed duplicated transforms (same location and rotation), this may cause some problem